In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import nibabel as nib
from time import time
from xml.etree import ElementTree as ET
import os
import pandas as pd
import sys
from scipy.interpolate import interp1d
from scipy.ndimage.filters import gaussian_filter
from scipy import signal
from scipy.ndimage import zoom
import smtplib
from email.mime.text import MIMEText
import re
import skimage
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import scipy
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
import h5py
import numpy_indexed as npi
from sklearn.linear_model import LassoLarsIC
from scipy.linalg import toeplitz


sys.path.insert(0, '/home/users/brezovec/.local/lib/python3.6/site-packages/lib/python/')
import ants

%matplotlib inline

from BigBadBrain.brain import bleaching_correction, z_score_brain, get_resolution, save_brain, load_numpy_brain, get_dims
from BigBadBrain.fictrac import load_fictrac, prep_fictrac
from BigBadBrain.utils import load_timestamps, get_fly_folders, send_email
from BigBadBrain.glm import fit_glm, save_glm_map
from BigBadBrain.motcorr import get_motcorr_brain
from BigBadBrain.visual import load_visual_stimuli_data, load_photodiode, parse_stim_starts_photodiode

In [2]:
root_path = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset'
vision_path = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/fly_28/visual'
folder = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/fly_28'

In [3]:
t,pd1,pd2 = load_photodiode(vision_path)
stimuli, unique_stimuli = load_visual_stimuli_data(vision_path)

loading photodiode data... done
loading visual stimuli data... done


In [4]:
stimuli_starts = parse_stim_starts_photodiode(pd1,stimuli)

Successfully parsed all stimuli from photodiode output.


In [5]:
### Add times list to each unique stimulus
for unique_stimulus in unique_stimuli:
    unique_stimulus['times'] = [stimuli_starts[i] for i, x in enumerate(stimuli) if x == unique_stimulus]

In [242]:
unique_stimuli

[{'angle': 0,
  'color': 1,
  'name': 'SineGrating',
  'period': 20,
  'rate': 20,
  'times': [182094.7,
   200746.1,
   216694.6,
   223021.9,
   235893.7,
   245393.1,
   257795.6,
   276977.3,
   289831.7,
   305962.6,
   309108.9,
   324996.5,
   337868.3,
   357388.7,
   360639.4,
   382584.8,
   395256.7,
   398402.9,
   600787.8,
   613955.0,
   623628.4,
   633075.7,
   645956.2,
   655890.4,
   675428.4,
   688282.7,
   701180.6,
   717268.1,
   730278.9,
   739865.3,
   752893.6,
   765643.6,
   781931.0,
   791378.4,
   801130.2,
   810829.5,
   1011267.6,
   1023991.6,
   1036559.2,
   1046406.4,
   1055940.7,
   1075078.9,
   1091140.3,
   1094147.5,
   1110278.5,
   1119769.4,
   1141923.3,
   1145304.3,
   1168249.3,
   1171412.9,
   1187300.4,
   1203474.9,
   1216398.8,
   1222543.6,
   1421339.0,
   1427509.8,
   1443240.9,
   1455739.0,
   1465325.3,
   1487444.7,
   1493615.4,
   1503688.6,
   1526051.3,
   1532535.0,
   1542112.8,
   1564527.6,
   1567534.8,
   158

In [6]:
timestamps = load_timestamps(folder)


~~ Loading Timestamps ~~
Trying to load timestamp data from hdf5 file.
Success.
Duration: 73.78 ms


In [7]:
zbrain_file = os.path.join(folder, 'brain_zscored_green.nii')
brain = load_numpy_brain(zbrain_file)

In [8]:
dims = get_dims(brain)

In [9]:
brain.shape

(129, 80, 51, 4493)

In [ ]:
# y=Xb
# y is a vector of a voxel's activity
# X is a toeplitz of when a stimulus occurs


# Each stim gets 20 bins, each bin is 100ms wide.
# bins start 500ms before stim, last 1000ms of stim, and 500ms at end
# For each stim, go through a voxel's activity and put in appropriate bin based on timestamps
# If 72 presentations, that is 72000ms of "ON" and if 400ms imaging rate, that is 72000/400 = 180 timepoints
# with 18 timepoints per 100ms bin. Then use that for GLM. That might work.

In [115]:
y=0
x=0
z=0
voxel_activity = brain[y,x,z,:]
voxel_times = timestamps[:,z]

In [ ]:
times[-1]

In [22]:
len(voxel_activity)

4493

In [220]:
len(unique_stimuli[0]['times'])

0

In [119]:
unique_stimuli[0]['angle']

180

In [111]:
bin_size = 100 #in ms
pre_dur = -500 #in ms
post_dur = 1500 #in ms

bins = create_bins(bin_size,pre_dur,post_dur)
stimuli_times = unique_stimuli[0]['times']

for z in range(dims['z']):

        ### Printing updates ###
        if z == dims['z']-1:
            print('{}.'.format(z))
            sys.stdout.flush()
        else:
            print('{}, '.format(z), end = '')
            sys.stdout.flush()

        voxel_times = timestamps[:,z]
        X = create_visual_X(stimuli_times, voxel_times, bins)
        
        for x in range(dims['x']):
            for y in range(dims['y']):
                Y = brain[y,x,z,:]
                model = LassoLarsIC(criterion='bic')
                model.fit(X, Y)
                betas.append(model.coef_)
                scores.append(model.score(X,Y))
    scores = np.reshape(scores, (dims['z'], dims['x'], dims['y']))
    betas = np.reshape(betas, (dims['z'], dims['x'], dims['y'], beta_len))
    return scores, betas

In [116]:
bins = create_bins(bin_size,pre_dur,post_dur)
stimuli_times = unique_stimuli[0]['times']
X = create_visual_X(stimuli_times, voxel_times, bins)

In [113]:
def create_bins(bin_size,pre_dur,post_dur):
    bins_pre = np.flip(np.arange(0,pre_dur-1,-bin_size),axis=0)
    bins_post = np.arange(0,post_dur+1,bin_size)
    bins = np.unique(np.concatenate((bins_pre, bins_post)))
    return bins

def create_visual_X(stimuli_times, voxel_times, bins):
    ### Get vector that assigns voxel activities to bins ###
    # Create real-time bins for each stimulus presentation
    time_bins = np.add.outer(stimuli_times,bins)
    time_bins_flat = np.reshape(time_bins,time_bins.shape[0]*time_bins.shape[1])
    # Find which bin each voxel timestamp belongs to
    binned = np.searchsorted(time_bins_flat, voxel_times)
    # The mod will give numbers that match bin numbers
    bin_mod = binned%21

    ### Use bin vector to create X matrix ###
    X = np.zeros((len(bin_mod), len(bins)))
    # This is used to correct index into X
    axis = np.arange(0,len(bin_mod))
    # For each row of X, put a 1 in the correct column (bin) if that row gets one
    X[axis,bin_mod] = 1
    # Remove first column, which is nonsense
    X = X[:,1:]
    return X

In [117]:
np.sum(X)

360.0

In [78]:
#binned_activity = npi.group_by(bin_mod, voxel_activity)[1][1:]

In [79]:
X.shape

(4493, 20)

In [88]:
np.sum(X)

360.0

In [95]:
np.sum(bin_mod>0)

360

In [253]:
len(bins)

21

In [244]:
for i in range(20):
    print(len(binned_activity[i]))

23
16
18
15
23
16
17
16
23
15
18
16
23
15
18
16
23
15
18
16


In [254]:
row = np.zeros(len(bins)-1)
row[0] = 1
X = toeplitz(row)
X = np.tile(X,(num_voxel_to_avg,1))

In [255]:
X

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [216]:
binned_activity

array([], shape=(0, 4493), dtype=float64)

In [160]:
80*128*50/1000/60

8.533333333333333

In [134]:
test = {voxel_activity: bin_mod}

TypeError: unhashable type: 'numpy.ndarray'

In [135]:
[0,0,0,0,0,1,0,0,2,2,0,1,1,0,0,0]

[0, 0, 0, 0, 0, 1, 0, 0, 2, 2, 0, 1, 1, 0, 0, 0]

In [136]:
np.where(voxel_activity, bin_mod)

ValueError: either both or neither of x and y should be given

In [142]:
t0= time()
[x for i,x in enumerate(voxel_activity) if bin_mod[i] == 1]
print(time()-t0)

0.0023462772369384766


In [158]:
t0=time()
test = npi.group_by(bin_mod, voxel_activity)[1]
print(time()-t0)

0.0008833408355712891


In [184]:
len(test[1][20])

16

In [133]:
bin_mod[voxel_activity]

IndexError: arrays used as indices must be of integer (or boolean) type

In [129]:
binned_activity = bin_mod

array([0, 0, 0, ..., 0, 0, 0])

In [48]:
t0 = time()
bin_set = bins+unique_stimuli[0]['times'][0]
#[x for x in np.searchsorted(bin_set, times) if x > 0 and x < 21]
print(time()-t0)

0.00010919570922851562


In [57]:
t0=time()
time_bins = np.add.outer(unique_stimuli[0]['times'],bins)
print(time()-t0)

0.00013256072998046875


In [53]:
(0.00010919570922851562)*120*50*80

52.4139404296875

In [66]:
time_bins_flat = np.reshape(time_bins,time_bins.shape[0]*time_bins.shape[1])

In [99]:
t0=time()
binned = np.searchsorted(time_bins_flat, times)
print(time()-t0)

0.0003504753112792969


In [124]:
t0=time()
bin_mod = binned%21
test = [x for x in bin_mod if x>0]
print(time()-t0)

0.0009832382202148438


In [119]:
test

[1,
 5,
 9,
 13,
 17,
 2,
 6,
 11,
 15,
 19,
 3,
 7,
 11,
 15,
 19,
 4,
 8,
 12,
 16,
 20,
 4,
 8,
 12,
 16,
 20,
 1,
 5,
 9,
 13,
 17,
 1,
 5,
 9,
 13,
 17,
 1,
 5,
 9,
 13,
 17,
 1,
 5,
 9,
 13,
 17,
 4,
 8,
 12,
 16,
 20,
 1,
 5,
 9,
 13,
 17,
 2,
 6,
 10,
 14,
 18,
 2,
 6,
 10,
 14,
 18,
 3,
 7,
 11,
 15,
 19,
 2,
 6,
 10,
 14,
 18,
 3,
 7,
 11,
 15,
 19,
 1,
 5,
 9,
 13,
 17,
 1,
 5,
 9,
 13,
 17,
 1,
 5,
 9,
 13,
 17,
 1,
 5,
 9,
 13,
 17,
 1,
 5,
 9,
 13,
 17,
 2,
 6,
 10,
 14,
 18,
 2,
 6,
 10,
 14,
 18,
 3,
 7,
 11,
 15,
 19,
 4,
 8,
 12,
 16,
 20,
 3,
 7,
 11,
 15,
 19,
 2,
 6,
 10,
 14,
 18,
 2,
 6,
 10,
 14,
 18,
 4,
 8,
 12,
 16,
 20,
 4,
 8,
 12,
 16,
 20,
 2,
 6,
 10,
 14,
 18,
 3,
 7,
 11,
 15,
 19,
 4,
 8,
 12,
 16,
 20,
 2,
 6,
 10,
 14,
 18,
 1,
 5,
 9,
 13,
 17,
 4,
 8,
 12,
 16,
 20,
 3,
 7,
 11,
 15,
 19,
 4,
 8,
 12,
 16,
 20,
 2,
 6,
 10,
 14,
 18,
 4,
 8,
 12,
 16,
 20,
 1,
 5,
 9,
 13,
 17,
 2,
 6,
 10,
 14,
 18,
 1,
 5,
 9,
 13,
 17,
 3,
 7,
 11,
 15,
 19,
 2

In [76]:

binned = np.reshape(binned, (time_bins.shape[0],time_bins.shape[1]))

ValueError: cannot reshape array of size 4493 into shape (72,21)

In [86]:
t0=time()
len(np.unique(binned))
#print(time()-t0)

433

In [ ]:
# step 1: pull out times that are between bin start and end
# step 2: parse those into 100ms bins



In [102]:
len(bin_set)

21

In [113]:
bins

array([-500, -400, -300, -200, -100,    0,  100,  200,  300,  400,  500,
        600,  700,  800,  900, 1000, 1100, 1200, 1300, 1400, 1500])

In [104]:
time_bins

array([[ 181594.7,  181694.7,  181794.7, ...,  183394.7,  183494.7,
         183594.7],
       [ 200246.1,  200346.1,  200446.1, ...,  202046.1,  202146.1,
         202246.1],
       [ 216194.6,  216294.6,  216394.6, ...,  217994.6,  218094.6,
         218194.6],
       ...,
       [1605597.9, 1605697.9, 1605797.9, ..., 1607397.9, 1607497.9,
        1607597.9],
       [1621555.1, 1621655.1, 1621755.1, ..., 1623355.1, 1623455.1,
        1623555.1],
       [1634765.7, 1634865.7, 1634965.7, ..., 1636565.7, 1636665.7,
        1636765.7]])

In [8]:
len(unique_stimuli[0]['times'])

72